<a href="https://colab.research.google.com/github/tensorflow/tpu/blob/master/tools/colab/bert_finetuning_with_cloud_tpus.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# BERT Multiclass Prediction for CFPB Data: Issues from Narratives


### Set up TPU environment

In [0]:
import datetime
import json
import os
import pprint
import random
import string
import sys
import tensorflow as tf
import pandas as pd
from sklearn.model_selection import train_test_split
import time

assert 'COLAB_TPU_ADDR' in os.environ, 'ERROR: Not connected to a TPU runtime; please see the first cell in this notebook for instructions!'
TPU_ADDRESS = 'grpc://' + os.environ['COLAB_TPU_ADDR']
print('TPU address is', TPU_ADDRESS)

from google.colab import auth
auth.authenticate_user()
with tf.Session(TPU_ADDRESS) as session:
  print('TPU devices:')
  pprint.pprint(session.list_devices())

  # Upload credentials to TPU.
  with open('/content/adc.json', 'r') as f:
    auth_info = json.load(f)
  tf.contrib.cloud.configure_gcs(session, credentials=auth_info)
  # Now credentials are set for all future sessions on this TPU.

TPU address is grpc://10.49.235.234:8470
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

TPU devices:
[_DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:CPU:0, CPU, -1, 9692817087067498932),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 17179869184, 3894156805154206250),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:0, TPU, 17179869184, 11042007275963063716),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:1, TPU, 17179869184, 14274637910424650287),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:2, TPU, 17179869184, 13193592791010722168),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:

### Prepare and import BERT modules


In [0]:
import sys

!test -d bert_repo || git clone https://github.com/google-research/bert bert_repo
if not 'bert_repo' in sys.path:
  sys.path += ['bert_repo']

# import python modules defined by BERT
import modeling
import optimization
import run_classifier
import run_classifier_with_tfhub
import tokenization

# import tfhub 
import tensorflow_hub as hub

## Load Data, Preprocess, and Split

In [0]:
from google.colab import drive
drive.mount('/content/drive/')


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive/


In [0]:
folder = 'drive/My Drive/IBM/'
datasetsfolder = 'drive/My Drive/IBM/datasets/'

df = pd.read_csv(folder+'Consumer_Complaints2.csv', usecols=['Consumer complaint narrative','Issue2'])
df = df.dropna(subset=['Consumer complaint narrative'])

In [0]:
listofxs = ["X"*2, "X"*3, "X"*4, "X"*5, "X"*6, "X"*7, "X"*8, "X"*9, "X"*10, "X"*11, "XX/XX", "XX/XX/XXX", "XX/XX/XX", "XX/XXXX"]

for xs in listofxs:
  df['Consumer complaint narrative'] = df['Consumer complaint narrative'].replace(to_replace = xs, value = '', regex = True)

df['Consumer complaint narrative'] = df['Consumer complaint narrative'].replace(to_replace = '//', value = '', regex = True)
df['Consumer complaint narrative'] = df['Consumer complaint narrative'].replace(to_replace = '{', value = '', regex = True)
df['Consumer complaint narrative'] = df['Consumer complaint narrative'].replace(to_replace = '}', value = '', regex = True)
df['Consumer complaint narrative'] = df['Consumer complaint narrative'].replace(to_replace = ' +', value = ' ', regex = True)

In [0]:
df = df[df['Issue2']!='Other']

In [0]:
issues_considered = list(df.Issue2.value_counts().index)

df.shape

(436390, 2)

In [0]:
df_bert = pd.DataFrame({'id': df.index,
    'label': df['Issue2'],
    'alpha': ['a']*df.shape[0],
    'text': df['Consumer complaint narrative']})

In [0]:
df_train, df_test = train_test_split(df_bert, test_size=0.1)

df_test_labels = df_test['label']
df_test = pd.DataFrame({'id': df_test.index,
    'text': df_test['text']})

df_train, df_dev = train_test_split(df_train, test_size=0.115)

#Limit training size to 150,000
df_train = df_train.sample(150000)

In [0]:
df_train.head()

,id,label,alpha,text
70941,70941,Incorrect information on your report,a,I have been a victim of identity theft so I re...
414195,414195,Attempts to collect debt not owed,a,Greetings ; Portfolio Recovery is indicating e...
2136,2136,Incorrect information on your report,a,I have NEVER had an account with or as far as ...
293711,293711,Dealing with my lender or servicer,a,We have been paying on a $6000.00 loan since a...
252778,252778,Communication tactics,a,"I do n't know who they are, they call and when..."


In [0]:
df_train.to_csv(datasetsfolder + 'train.tsv', sep='\t', index=False, header=False)
df_dev.to_csv(datasetsfolder + 'dev.tsv', sep='\t', index=False, header=False)
df_test.to_csv(datasetsfolder + 'test.tsv', sep='\t', index=False, header=True)

### Prepare for training



In [0]:
# I'll take CoLA task as a template
TASK = 'CoLA'
TASK_DATA_DIR = datasetsfolder

BUCKET = 'bertissue' #@param {type:"string"}
assert BUCKET, 'Must specify an existing GCS bucket name'
OUTPUT_DIR = 'gs://{}/bert-tfhub/models/{}'.format(BUCKET, TASK)
tf.gfile.MakeDirs(OUTPUT_DIR)
print('***** Model output directory: {} *****'.format(OUTPUT_DIR))

# I'm using uncased_L-12_H-768_A-12. The bigger model will probably obtain better results.
BERT_MODEL = 'uncased_L-12_H-768_A-12'
BERT_MODEL_HUB = 'https://tfhub.dev/google/bert_' + BERT_MODEL + '/1'

#BERT_MODEL = 'albert_base'
#BERT_MODEL_HUB = 'https://tfhub.dev/google/' + BERT_MODEL + '/2'



***** Model output directory: gs://bertissue/bert-tfhub/models/CoLA *****


Hyperprams:

In [0]:
# Hyperparamethers
TRAIN_BATCH_SIZE = 32
EVAL_BATCH_SIZE = 8
PREDICT_BATCH_SIZE = 8
LEARNING_RATE = 2e-5
NUM_TRAIN_EPOCHS = 3.0
MAX_SEQ_LENGTH = 384
WARMUP_PROPORTION = 0.1

# Model configs
SAVE_CHECKPOINTS_STEPS = 1000
SAVE_SUMMARY_STEPS = 500

In [0]:
tokenizer = run_classifier_with_tfhub.create_tokenizer_from_hub_module('https://tfhub.dev/google/bert_uncased_L-12_H-768_A-12/1')

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


In [0]:
import csv

class InputExample(object):
  """A single training/test example for simple sequence classification."""

  def __init__(self, guid, text_a, text_b=None, label=None):
    """Constructs a InputExample.
    Args:
      guid: Unique id for the example.
      text_a: string. The untokenized text of the first sequence. For single
        sequence tasks, only this sequence must be specified.
      text_b: (Optional) string. The untokenized text of the second sequence.
        Only must be specified for sequence pair tasks.
      label: (Optional) string. The label of the example. This should be
        specified for train and dev examples, but not for test examples.
    """
    self.guid = guid
    self.text_a = text_a
    self.text_b = text_b
    self.label = label

class DataProcessor(object):
  """Base class for data converters for sequence classification data sets."""

  def get_train_examples(self, data_dir):
    """Gets a collection of `InputExample`s for the train set."""
    raise NotImplementedError()

  def get_dev_examples(self, data_dir):
    """Gets a collection of `InputExample`s for the dev set."""
    raise NotImplementedError()

  def get_test_examples(self, data_dir):
    """Gets a collection of `InputExample`s for prediction."""
    raise NotImplementedError()

  def get_labels(self):
    """Gets the list of labels for this data set."""
    raise NotImplementedError()

  @classmethod
  def _read_tsv(cls, input_file):
    """Reads a tab separated value file."""
    with tf.gfile.Open(input_file, "r") as f:
      reader = csv.reader(f, delimiter="\t")
      lines = []
      for line in reader:
        lines.append(line)
      return lines

class ModifiedProcessor(DataProcessor):

  def get_train_examples(self, data_dir):
    """See base class."""
    return self._create_examples(
        self._read_tsv(os.path.join(data_dir, "train.tsv")), "train")

  def get_dev_examples(self, data_dir):
    """See base class."""
    return self._create_examples(
        self._read_tsv(os.path.join(data_dir, "dev.tsv")), "dev")

  def get_test_examples(self, data_dir):
    """See base class."""
    return self._create_examples(
        self._read_tsv(os.path.join(data_dir, "test.tsv")), "test")

  def get_labels(self, products_considered):
    """See base class."""
    return products_considered

  def _create_examples(self, lines, set_type):
    """Creates examples for the training and dev sets."""
    examples = []
    for (i, line) in enumerate(lines):
      # Only the test set has a header
      if set_type == "test" and i == 0:
        continue
      guid = "%s-%s" % (set_type, i)
      if set_type == "test":
        text_a = tokenization.convert_to_unicode(line[1])
        label = "0"
      else:
        text_a = tokenization.convert_to_unicode(line[3])
        label = tokenization.convert_to_unicode(line[1])
      examples.append(
          InputExample(guid=guid, text_a=text_a, text_b=None, label=label))
    return examples


In [0]:
processor = ModifiedProcessor()
label_list = processor.get_labels(issues_considered)
train_examples = processor.get_train_examples(TASK_DATA_DIR)

#label_list = products_considered
#train_examples = create_examples(train)

In [0]:
# Compute number of train and warmup steps from batch size
num_train_steps = int(len(train_examples) / TRAIN_BATCH_SIZE * NUM_TRAIN_EPOCHS)
num_warmup_steps = int(num_train_steps * WARMUP_PROPORTION)

# Setup TPU related config
tpu_cluster_resolver = tf.contrib.cluster_resolver.TPUClusterResolver(TPU_ADDRESS)
NUM_TPU_CORES = 8
ITERATIONS_PER_LOOP = 1000

def get_run_config(output_dir):
  return tf.contrib.tpu.RunConfig(
    cluster=tpu_cluster_resolver,
    model_dir=output_dir,
    save_checkpoints_steps=SAVE_CHECKPOINTS_STEPS,
    tpu_config=tf.contrib.tpu.TPUConfig(
        iterations_per_loop=ITERATIONS_PER_LOOP,
        num_shards=NUM_TPU_CORES,
        per_host_input_for_training=tf.contrib.tpu.InputPipelineConfig.PER_HOST_V2))

# Fine-tune and Run Predictions on a pretrained BERT Model from TF Hub

In [0]:
# Force TF Hub writes to the GS bucket we provide.
os.environ['TFHUB_CACHE_DIR'] = OUTPUT_DIR

model_fn = run_classifier_with_tfhub.model_fn_builder(
  num_labels=len(label_list),
  learning_rate=LEARNING_RATE,
  num_train_steps=num_train_steps,
  num_warmup_steps=num_warmup_steps,
  use_tpu=True,
  bert_hub_module_handle=BERT_MODEL_HUB
)

estimator_from_tfhub = tf.contrib.tpu.TPUEstimator(
  use_tpu=True,
  model_fn=model_fn,
  config=get_run_config(OUTPUT_DIR),
  train_batch_size=TRAIN_BATCH_SIZE,
  eval_batch_size=EVAL_BATCH_SIZE,
  predict_batch_size=PREDICT_BATCH_SIZE,
)


INFO:tensorflow:Using config: {'_model_dir': 'gs://bertissue/bert-tfhub/models/CoLA', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': 1000, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
cluster_def {
  job {
    name: "worker"
    tasks {
      key: 0
      value: "10.54.164.98:8470"
    }
  }
}
isolate_session_state: true
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': None, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f089b880128>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': 'grpc://10.54.164.98:8470', '_evaluation_master': 'grpc://10.54.164.98:8470', '_is_chief': True, '_num_ps_re

INFO:tensorflow:Using config: {'_model_dir': 'gs://bertissue/bert-tfhub/models/CoLA', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': 1000, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
cluster_def {
  job {
    name: "worker"
    tasks {
      key: 0
      value: "10.54.164.98:8470"
    }
  }
}
isolate_session_state: true
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': None, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f089b880128>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': 'grpc://10.54.164.98:8470', '_evaluation_master': 'grpc://10.54.164.98:8470', '_is_chief': True, '_num_ps_re

INFO:tensorflow:_TPUContext: eval_on_tpu True


INFO:tensorflow:_TPUContext: eval_on_tpu True


In [0]:
# Train the model
def model_train(estimator):
  print('MRPC/CoLA on BERT base model normally takes about 2-3 minutes. Please wait...')
  # We'll set sequences to be at most 128 tokens long.
  train_features = run_classifier.convert_examples_to_features(
      train_examples, label_list, MAX_SEQ_LENGTH, tokenizer)
  print('***** Started training at {} *****'.format(datetime.datetime.now()))
  print('  Num examples = {}'.format(len(train_examples)))
  print('  Batch size = {}'.format(TRAIN_BATCH_SIZE))
  tf.logging.info("  Num steps = %d", num_train_steps)
  train_input_fn = run_classifier.input_fn_builder(
      features=train_features,
      seq_length=MAX_SEQ_LENGTH,
      is_training=True,
      drop_remainder=True)
  estimator.train(input_fn=train_input_fn, max_steps=num_train_steps)
  print('***** Finished training at {} *****'.format(datetime.datetime.now()))


In [0]:
start = time.time()
model_train(estimator_from_tfhub)
print(time.time() - start)

MRPC/CoLA on BERT base model normally takes about 2-3 minutes. Please wait...



INFO:tensorflow:Writing example 0 of 150000


INFO:tensorflow:Writing example 0 of 150000


INFO:tensorflow:*** Example ***


INFO:tensorflow:*** Example ***


...

INFO:tensorflow:Loss for final step: 0.24308255.


INFO:tensorflow:Loss for final step: 0.24308255.


INFO:tensorflow:training_loop marked as finished


INFO:tensorflow:training_loop marked as finished


***** Finished training at 2019-12-03 21:18:53.725060 *****
3505.4762723445892


In [0]:
def model_eval(estimator):
  # Eval the model.
  eval_examples = processor.get_dev_examples(TASK_DATA_DIR)
  eval_features = run_classifier.convert_examples_to_features(
      eval_examples, label_list, MAX_SEQ_LENGTH, tokenizer)
  print('***** Started evaluation at {} *****'.format(datetime.datetime.now()))
  print('  Num examples = {}'.format(len(eval_examples)))
  print('  Batch size = {}'.format(EVAL_BATCH_SIZE))

  # Eval will be slightly WRONG on the TPU because it will truncate
  # the last batch.
  eval_steps = int(len(eval_examples) / EVAL_BATCH_SIZE)
  eval_input_fn = run_classifier.input_fn_builder(
      features=eval_features,
      seq_length=MAX_SEQ_LENGTH,
      is_training=False,
      drop_remainder=True)
  result = estimator.evaluate(input_fn=eval_input_fn, steps=eval_steps)
  print('***** Finished evaluation at {} *****'.format(datetime.datetime.now()))
  output_eval_file = os.path.join(OUTPUT_DIR, "eval_results.txt")
  with tf.gfile.GFile(output_eval_file, "w") as writer:
    print("***** Eval results *****")
    for key in sorted(result.keys()):
      print('  {} = {}'.format(key, str(result[key])))
      writer.write("%s = %s\n" % (key, str(result[key])))


In [0]:
model_eval(estimator_from_tfhub)

INFO:tensorflow:Writing example 0 of 45167


INFO:tensorflow:Writing example 0 of 45167


INFO:tensorflow:*** Example ***


INFO:tensorflow:*** Example ***


INFO:tensorflow:guid: dev-0


...

INFO:tensorflow:Saving 'checkpoint_path' summary for global step 14062: gs://bertissue/bert-tfhub/models/CoLA/model.ckpt-14062


INFO:tensorflow:Saving 'checkpoint_path' summary for global step 14062: gs://bertissue/bert-tfhub/models/CoLA/model.ckpt-14062


INFO:tensorflow:evaluation_loop marked as finished


INFO:tensorflow:evaluation_loop marked as finished


***** Finished evaluation at 2019-12-03 21:23:25.293698 *****
***** Eval results *****
  eval_accuracy = 0.65566874
  eval_loss = 1.2625928
  global_step = 14062
  loss = 1.2826482


In [0]:
def model_predict(estimator):
  # Make predictions on a subset of eval examples
  prediction_examples = processor.get_dev_examples(TASK_DATA_DIR)
  input_features = run_classifier.convert_examples_to_features(prediction_examples, label_list, MAX_SEQ_LENGTH, tokenizer)
  predict_input_fn = run_classifier.input_fn_builder(features=input_features, seq_length=MAX_SEQ_LENGTH, is_training=False, drop_remainder=True)
  predictions = estimator.predict(predict_input_fn)

  probabilities = []
  for example, prediction in zip(prediction_examples, predictions):
    print('text_a: %s\ntext_b: %s\nlabel:%s\nprediction:%s\n' % (example.text_a, example.text_b, str(example.label), prediction['probabilities']))
    probabilities.append(prediction["probabilities"])
  return probabilities


In [0]:
#import numpy as np

In [0]:
#predicted_values = [np.argmax(pred) for pred in predictions]

In [0]:
#dictvalues = dict(zip(products_considered, range(0,len(products_considered))))
#df_test_labels.replace(dictvalues, inplace=True)

In [0]:
#from sklearn.metrics import accuracy_score

In [0]:
#accuracy_score(predicted_values, df_test_labels.to_numpy())

In [0]:
#predicted_values[0:10]

In [0]:
#df_test_labels.to_numpy()[0:10]

# Fine-tune and run predictions on a pre-trained BERT model from checkpoints

Alternatively, you can also load pre-trained BERT models from saved checkpoints.

In [0]:
# Setup task specific model and TPU running config.
#BERT_PRETRAINED_DIR = 'gs://cloud-tpu-checkpoints/bert/' + BERT_MODEL 
#print('***** BERT pretrained directory: {} *****'.format(BERT_PRETRAINED_DIR))
#!gsutil ls $BERT_PRETRAINED_DIR

#CONFIG_FILE = os.path.join(BERT_PRETRAINED_DIR, 'bert_config.json')
#INIT_CHECKPOINT = os.path.join(BERT_PRETRAINED_DIR, 'bert_model.ckpt')

#model_fn = run_classifier.model_fn_builder(
#  bert_config=modeling.BertConfig.from_json_file(CONFIG_FILE),
#  num_labels=len(label_list),
#  init_checkpoint=INIT_CHECKPOINT,
#  learning_rate=LEARNING_RATE,
#  num_train_steps=num_train_steps,
#  num_warmup_steps=num_warmup_steps,
#  use_tpu=True,
#  use_one_hot_embeddings=True
#)

#OUTPUT_DIR = OUTPUT_DIR.replace('bert-tfhub', 'bert-checkpoints')
#tf.gfile.MakeDirs(OUTPUT_DIR)

#estimator_from_checkpoints = tf.contrib.tpu.TPUEstimator(
#  use_tpu=True,
#  model_fn=model_fn,
#  config=get_run_config(OUTPUT_DIR),
#  train_batch_size=TRAIN_BATCH_SIZE,
#  eval_batch_size=EVAL_BATCH_SIZE,
#  predict_batch_size=PREDICT_BATCH_SIZE,
#)

Now, you can repeat the training, evaluation, and prediction steps.

In [0]:
#model_train(estimator_from_checkpoints)

In [0]:
#model_eval(estimator_from_checkpoints)

In [0]:
#model_predict(estimator_from_checkpoints)